<h3>Introduction.</h3>
<p>The goal of the notebook is to predict the sale price of a particular piece of heavy equiment at auction based on it's usage, equipment type, and configuaration.  The data is sourced from auction result postings and includes information on usage and equipment configurations.In our quest we will be learning how to use Random forest. </p>

In [2]:
%load_ext autoreload 
#  reloads modules automatically before entering 
# the execution of code typed at the IPython prompt.
%autoreload 2

%matplotlib inline 
# draw the plots in the notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# importing the libraries we're going to be using.
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

ModuleNotFoundError: No module named 'fastai'